# Topics in Python
## Final Project

### Cody Coleman
### Due: May 20th, 2025

The dataset was taken from Kaggle.com The link to the dataset is right below this.

https://www.kaggle.com/datasets/khushikyad001/impact-of-screen-time-on-mental-health/data

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint # Maybe get to this
from tensorflow.keras.metrics import AUC, Precision, Recall, BinaryAccuracy
from tensorflow.keras.datasets import fashion_mnist ## IMPORT THIS
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, root_mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR, SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [2]:
data = pd.read_csv("digital_diet_mental_health.csv")
SEED = 1218727

In [3]:
data.head()

,user_id,age,gender,daily_screen_time_hours,phone_usage_hours,laptop_usage_hours,tablet_usage_hours,tv_usage_hours,social_media_hours,work_related_hours,...,stress_level,physical_activity_hours_per_week,location_type,mental_health_score,uses_wellness_apps,eats_healthy,caffeine_intake_mg_per_day,weekly_anxiety_score,weekly_depression_score,mindfulness_minutes_per_day
0,user_1,51,Female,4.8,3.4,1.3,1.6,1.6,4.1,2.0,...,10,0.7,Urban,32,1,1,125.2,13,15,4.0
1,user_2,64,Male,3.9,3.5,1.8,0.9,2.0,2.7,3.1,...,6,4.3,Suburban,75,0,1,150.4,19,18,6.5
2,user_3,41,Other,10.5,2.1,2.6,0.7,2.2,3.0,2.8,...,5,3.1,Suburban,22,0,0,187.9,7,3,6.9
3,user_4,27,Other,8.8,0.0,0.0,0.7,2.5,3.3,1.6,...,5,0.0,Rural,22,0,1,73.6,7,2,4.8
4,user_5,55,Male,5.9,1.7,1.1,1.5,1.6,1.1,3.6,...,7,3.0,Urban,64,1,1,217.5,8,10,0.0


In [3]:
data = data.drop("user_id", axis=1)

In [5]:
data.dtypes

age                                   int64
gender                               object
daily_screen_time_hours             float64
phone_usage_hours                   float64
laptop_usage_hours                  float64
tablet_usage_hours                  float64
tv_usage_hours                      float64
social_media_hours                  float64
work_related_hours                  float64
entertainment_hours                 float64
gaming_hours                        float64
sleep_duration_hours                float64
sleep_quality                         int64
mood_rating                           int64
stress_level                          int64
physical_activity_hours_per_week    float64
location_type                        object
mental_health_score                   int64
uses_wellness_apps                    int64
eats_healthy                          int64
caffeine_intake_mg_per_day          float64
weekly_anxiety_score                  int64
weekly_depression_score         

In [4]:
data["location_type"].unique()

array(['Urban', 'Suburban', 'Rural'], dtype=object)

In [5]:
def gender(x):
    if x == "Female":
        return 0
    if x == "Male":
        return 1
    return 2

def location(x):
    if x == "Urban":
        return 0
    if x == "Suburban":
        return 1
    return 2

data.drop(["location_type", "gender"], axis=1)

data["gender"] = data["gender"].apply(gender)
data["location_type"] = data["location_type"].apply(location)

In [6]:
data.head()

,age,gender,daily_screen_time_hours,phone_usage_hours,laptop_usage_hours,tablet_usage_hours,tv_usage_hours,social_media_hours,work_related_hours,entertainment_hours,...,stress_level,physical_activity_hours_per_week,location_type,mental_health_score,uses_wellness_apps,eats_healthy,caffeine_intake_mg_per_day,weekly_anxiety_score,weekly_depression_score,mindfulness_minutes_per_day
0,51,0,4.8,3.4,1.3,1.6,1.6,4.1,2.0,1.0,...,10,0.7,0,32,1,1,125.2,13,15,4.0
1,64,1,3.9,3.5,1.8,0.9,2.0,2.7,3.1,1.0,...,6,4.3,1,75,0,1,150.4,19,18,6.5
2,41,2,10.5,2.1,2.6,0.7,2.2,3.0,2.8,4.1,...,5,3.1,1,22,0,0,187.9,7,3,6.9
3,27,2,8.8,0.0,0.0,0.7,2.5,3.3,1.6,1.3,...,5,0.0,2,22,0,1,73.6,7,2,4.8
4,55,1,5.9,1.7,1.1,1.5,1.6,1.1,3.6,0.8,...,7,3.0,0,64,1,1,217.5,8,10,0.0


In [7]:
data.drop(["gender"], axis=1)

,age,daily_screen_time_hours,phone_usage_hours,laptop_usage_hours,tablet_usage_hours,tv_usage_hours,social_media_hours,work_related_hours,entertainment_hours,gaming_hours,...,stress_level,physical_activity_hours_per_week,location_type,mental_health_score,uses_wellness_apps,eats_healthy,caffeine_intake_mg_per_day,weekly_anxiety_score,weekly_depression_score,mindfulness_minutes_per_day
0,51,4.8,3.4,1.3,1.6,1.6,4.1,2.0,1.0,1.7,...,10,0.7,0,32,1,1,125.2,13,15,4.0
1,64,3.9,3.5,1.8,0.9,2.0,2.7,3.1,1.0,1.5,...,6,4.3,1,75,0,1,150.4,19,18,6.5
2,41,10.5,2.1,2.6,0.7,2.2,3.0,2.8,4.1,1.7,...,5,3.1,1,22,0,0,187.9,7,3,6.9
3,27,8.8,0.0,0.0,0.7,2.5,3.3,1.6,1.3,0.4,...,5,0.0,2,22,0,1,73.6,7,2,4.8
4,55,5.9,1.7,1.1,1.5,1.6,1.1,3.6,0.8,0.8,...,7,3.0,0,64,1,1,217.5,8,10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,58,5.6,4.0,2.5,0.3,1.5,1.1,1.2,2.1,0.8,...,9,0.0,0,62,0,1,164.9,20,17,4.9
1996,62,3.9,3.1,1.0,1.5,1.1,2.7,4.1,6.4,1.8,...,8,2.7,0,29,0,0,172.6,15,15,25.5
1997,64,7.4,3.0,0.0,1.4,0.9,0.8,2.6,4.1,2.5,...,4,6.5,0,54,1,0,101.3,1,20,9.5
1998,19,4.2,4.4,2.3,0.9,1.4,1.7,1.2,2.0,0.7,...,8,2.6,0,28,0,0,123.7,1,11,13.4


In [8]:
data.columns

Index(['age', 'gender', 'daily_screen_time_hours', 'phone_usage_hours',
       'laptop_usage_hours', 'tablet_usage_hours', 'tv_usage_hours',
       'social_media_hours', 'work_related_hours', 'entertainment_hours',
       'gaming_hours', 'sleep_duration_hours', 'sleep_quality', 'mood_rating',
       'stress_level', 'physical_activity_hours_per_week', 'location_type',
       'mental_health_score', 'uses_wellness_apps', 'eats_healthy',
       'caffeine_intake_mg_per_day', 'weekly_anxiety_score',
       'weekly_depression_score', 'mindfulness_minutes_per_day'],
      dtype='object')

In [23]:
cb = tf.keras.callbacks.EarlyStopping(patience=2)

# Daily Screen Time Hours

In [11]:
y = data["daily_screen_time_hours"]
x = data.drop("daily_screen_time_hours", axis=1)

In [12]:
x.head()

,age,gender,phone_usage_hours,laptop_usage_hours,tablet_usage_hours,tv_usage_hours,social_media_hours,work_related_hours,entertainment_hours,gaming_hours,...,stress_level,physical_activity_hours_per_week,location_type,mental_health_score,uses_wellness_apps,eats_healthy,caffeine_intake_mg_per_day,weekly_anxiety_score,weekly_depression_score,mindfulness_minutes_per_day
0,51,0,3.4,1.3,1.6,1.6,4.1,2.0,1.0,1.7,...,10,0.7,0,32,1,1,125.2,13,15,4.0
1,64,1,3.5,1.8,0.9,2.0,2.7,3.1,1.0,1.5,...,6,4.3,1,75,0,1,150.4,19,18,6.5
2,41,2,2.1,2.6,0.7,2.2,3.0,2.8,4.1,1.7,...,5,3.1,1,22,0,0,187.9,7,3,6.9
3,27,2,0.0,0.0,0.7,2.5,3.3,1.6,1.3,0.4,...,5,0.0,2,22,0,1,73.6,7,2,4.8
4,55,1,1.7,1.1,1.5,1.6,1.1,3.6,0.8,0.8,...,7,3.0,0,64,1,1,217.5,8,10,0.0


In [13]:
y.head()

0     4.8
1     3.9
2    10.5
3     8.8
4     5.9
Name: daily_screen_time_hours, dtype: float64

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=SEED)

# Decision Tree Regressor

In [ ]:
param_grid = {
    "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
    "max_depth": [2, 4, 5, 6],
}
gs = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid=param_grid
)
gs.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'poisson'],
                         'max_depth': [2, 4, 5, 6]})

In [10]:
preds = gs.predict(x_test)
root_mean_squared_error(y_test, preds)

1.966594646163352

In [11]:
ex = 102
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 6.8
Predicted value: 5.972869147659058


# TensorFlow

In [17]:
x.shape

(2000, 23)

https://www.tensorflow.org/api_docs/python/tf/keras/metrics/MeanAbsoluteError

In [21]:
input_shape = (23, )
output_shape = (10,)
metrics = ["mae"]
EPOCHS = 1000
BATCH_SIZE = 10


model = Sequential([
    Input(input_shape),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dense(1, activation="linear")
])

model.compile(optimizer="adam", loss="mae", metrics=metrics)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         6,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,361 (185.00 KB)

 Trainable params: 47,361 (185.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
cb = tf.keras.callbacks.EarlyStopping(patience=2)

In [23]:
model.fit(x_train,  y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=cb)

Epoch 1/1000
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - loss: 4.6266 - mae: 4.6266 - val_loss: 2.2955 - val_mae: 2.2955
Epoch 2/1000
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - loss: 2.2169 - mae: 2.2169 - val_loss: 2.1554 - val_mae: 2.1554
Epoch 3/1000
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - loss: 2.1466 - mae: 2.1466 - val_loss: 2.3907 - val_mae: 2.3907
Epoch 4/1000
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 1.8520 - mae: 1.8520 - val_loss: 1.6837 - val_mae: 1.6837
Epoch 5/1000
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - loss: 1.9455 - mae: 1.9455 - val_loss: 1.8558 - val_mae: 1.8558
Epoch 6/1000
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step - loss: 1.8213 - mae: 1.8213 - val_loss: 1.8851 - val_mae: 1.8851


In [24]:
preds = model.predict(x_test)
root_mean_squared_error(y_test, preds)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


2.321095103473814

In [25]:
ex = 7
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 3.7
Predicted value: [4.571913]


In [26]:
ex = 110
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 8.7
Predicted value: [5.243303]


# Support Vector Regressor

In [14]:
param_grid = {
    "kernel": ["linear"],
    "gamma": ["scale", "auto"],
    "C": [0.1, 1, 10]
}
gs = GridSearchCV(
    estimator=SVR(),
    param_grid=param_grid
)
gs.fit(x_train, y_train)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear']})

In [15]:
preds = gs.predict(x_test)
root_mean_squared_error(y_test, preds)

1.9744543754612025

In [16]:
ex = 102
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 6.8
Predicted value: 5.7818690568469835


In [17]:
ex = 50
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 7.7
Predicted value: 5.633341838906339


In [18]:
ex = 10
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 7.4
Predicted value: 6.226171207241344


# Age

In [9]:
data.head()

,age,gender,daily_screen_time_hours,phone_usage_hours,laptop_usage_hours,tablet_usage_hours,tv_usage_hours,social_media_hours,work_related_hours,entertainment_hours,...,stress_level,physical_activity_hours_per_week,location_type,mental_health_score,uses_wellness_apps,eats_healthy,caffeine_intake_mg_per_day,weekly_anxiety_score,weekly_depression_score,mindfulness_minutes_per_day
0,51,0,4.8,3.4,1.3,1.6,1.6,4.1,2.0,1.0,...,10,0.7,0,32,1,1,125.2,13,15,4.0
1,64,1,3.9,3.5,1.8,0.9,2.0,2.7,3.1,1.0,...,6,4.3,1,75,0,1,150.4,19,18,6.5
2,41,2,10.5,2.1,2.6,0.7,2.2,3.0,2.8,4.1,...,5,3.1,1,22,0,0,187.9,7,3,6.9
3,27,2,8.8,0.0,0.0,0.7,2.5,3.3,1.6,1.3,...,5,0.0,2,22,0,1,73.6,7,2,4.8
4,55,1,5.9,1.7,1.1,1.5,1.6,1.1,3.6,0.8,...,7,3.0,0,64,1,1,217.5,8,10,0.0


In [10]:
y = data["age"]
x = data.drop("age", axis=1)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=SEED)

# Descision Tree Regressor

In [12]:
param_grid = {
    "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
    "max_depth": [1, 3, 4, 5],
}
gs = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid=param_grid
)
gs.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': [1, 3, 4, 5]})

In [13]:
preds = gs.predict(x_test)
root_mean_squared_error(y_test, preds)

14.84519351313976

In [14]:
ex = 102
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 37
Predicted value: 37.57007376185459


In [15]:
ex = 90
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 56
Predicted value: 37.57007376185459


In [16]:
ex = 110
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 55
Predicted value: 37.57007376185459


In [17]:
ex = 10
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 33
Predicted value: 39.87864823348694


In [18]:
ex = 200
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 50
Predicted value: 39.87864823348694


In [19]:
ex = 55
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 49
Predicted value: 37.57007376185459


In [38]:
x.shape

(2000, 23)

# Tensorflow

In [46]:
input_shape = (23, )
output_shape = (10,)
metrics = ["mse"]
EPOCHS = 100
BATCH_SIZE = 100


model = Sequential([
    Input(input_shape),
    Dense(128, activation="relu"),
    Dense(10, activation="relu"),
    Dense(10, activation="relu"),
    Dense(1, activation="linear")
])

model.compile(optimizer="adam", loss="mae", metrics=metrics)

model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                │ (None, 128)            │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 10)             │         1,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,483 (17.51 KB)

 Trainable params: 4,483 (17.51 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
model.fit(x_train,  y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=cb)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32.9954 - mse: 1359.5247 - val_loss: 16.7210 - val_mse: 409.1074
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.1745 - mse: 327.2947 - val_loss: 14.5047 - val_mse: 315.1367
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.4194 - mse: 299.4214 - val_loss: 14.2258 - val_mse: 290.2170
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.8550 - mse: 274.7169 - val_loss: 13.9112 - val_mse: 275.9259
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.7410 - mse: 265.1490 - val_loss: 13.8184 - val_mse: 270.1201
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.8536 - mse: 272.8796 - val_loss: 13.7468 - val_mse: 265.8520
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.4104 - mse: 255.7742 - val_loss: 13.7318 - val_mse: 264.3792
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.6905 - mse: 265.5840 - val_loss: 13.7378 - val_mse: 264.1739
Epoch 9/100
16/

In [48]:
preds = model.predict(x_test)
root_mean_squared_error(y_test, preds)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


15.69661808013916

In [49]:
ex = 102
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 37
Predicted value: [39.57062]


In [50]:
ex = 50
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 63
Predicted value: [43.42427]


In [51]:
ex = 10
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 33
Predicted value: [43.77974]


In [52]:
ex = 90
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 56
Predicted value: [38.757366]


In [53]:
ex = 70
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 14
Predicted value: [36.785492]


In [54]:
ex = 10
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 33
Predicted value: [43.77974]


# Support Vector Regressor

In [15]:
param_grid = {
    "kernel": ["linear"],
    "gamma": ["scale", "auto"],
    "C": [0.5, 1.5, 10.5]
}
gs = GridSearchCV(
    estimator=SVR(),
    param_grid=param_grid
)
gs.fit(x_train, y_train)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.5, 1.5, 10.5], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear']})

In [16]:
preds = gs.predict(x_test)
root_mean_squared_error(y_test, preds)

1.974567057663765

In [17]:
ex = 9
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 5.2
Predicted value: 6.235715684326707


In [18]:
ex = 11
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 5.9
Predicted value: 5.778250744781318


In [19]:
ex = 100
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 7.6
Predicted value: 6.253989822460782


In [20]:
ex = 90
print(f"Real value: {y_test.iloc[ex]}")
print(f"Predicted value: {preds[ex]}")

Real value: 8.3
Predicted value: 5.987595367995156
